In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
import urllib
import speech_recognition as sr
import threading
import time
import import_ipynb
from Untitled12 import *

def connected(host="http://google.com"):
    try:
        urllib.request.urlopen(host)
        return True
    except:
        return False


    
    
class Ui_Function(object):
    def setupUi(self, Function):
        Function.setObjectName("Function")
        Function.resize(737, 600)
        self.centralwidget = QtWidgets.QWidget(Function)
        self.centralwidget.setObjectName("centralwidget")
        self.lineEdit = QtWidgets.QLineEdit(self.centralwidget)
        self.lineEdit.setGeometry(QtCore.QRect(50, 70, 601, 31))
        self.lineEdit.setObjectName("lineEdit")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(50, 30, 151, 17))
        self.label.setObjectName("label")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(320, 120, 67, 17))
        self.label_2.setObjectName("label_2")
        self.label_3 = QtWidgets.QLabel(self.centralwidget)
        self.label_3.setGeometry(QtCore.QRect(60, 150, 251, 21))
        self.label_3.setObjectName("label_3")
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(60, 180, 101, 25))
        self.pushButton.setObjectName("pushButton")

        self.textEdit = QtWidgets.QTextEdit(self.centralwidget)
        self.textEdit.setGeometry(QtCore.QRect(60, 320, 601, 231))
        self.textEdit.setObjectName("textEdit")
        self.label_4 = QtWidgets.QLabel(self.centralwidget)
        self.label_4.setGeometry(QtCore.QRect(60, 290, 141, 21))
        self.label_4.setObjectName("label_4")
        self.pushButton_2 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_2.setGeometry(QtCore.QRect(570, 110, 89, 25))
        self.pushButton_2.setObjectName("pushButton_2")
        
        #self.pushButton_2.clicked.connect(Untitled12.give_rec("Angel of Hope"))
        
        self.label_5 = QtWidgets.QLabel(self.centralwidget)
        self.label_5.setGeometry(QtCore.QRect(60, 220, 241, 21))
        self.label_5.setObjectName("label_5")
        self.comboBox = QtWidgets.QComboBox(self.centralwidget)
        self.comboBox.setGeometry(QtCore.QRect(400, 180, 251, 31))
        self.comboBox.setObjectName("comboBox")
        self.comboBox.currentIndexChanged.connect(self.changeText)
        
        Function.setCentralWidget(self.centralwidget)

        self.retranslateUi(Function)
        QtCore.QMetaObject.connectSlotsByName(Function)
        
    def changeText(self, index):
        self.lineEdit.setText(self.comboBox.itemText(index))
        
    def retranslateUi(self, Function):
        _translate = QtCore.QCoreApplication.translate
        Function.setWindowTitle(_translate("Function", "MainWindow"))
        self.label.setText(_translate("Function", "Enter the Book Name:"))
        self.label_2.setText(_translate("Function", "OR"))
        self.label_3.setText(
            _translate("Function", "Write the Book Name by your Voice:")
        )
        self.pushButton.setText(_translate("Function", "Click and Say"))
        self.label_4.setText(_translate("Function", "Recommendations:"))
        self.pushButton_2.setText(_translate("Function", "Clear"))
        self.label_5.setText(_translate("Function", "TextLabel"))


class SpeechWorker(QtCore.QObject):
    messageChanged = QtCore.pyqtSignal(str)
    itemsChanged = QtCore.pyqtSignal(list)

    def speech(self):
        threading.Thread(target=self._speech, daemon=True).start()

    def _speech(self):
        if connected():
            r = sr.Recognizer()
            with sr.Microphone() as source:
                r.adjust_for_ambient_noise(source)
                self.messageChanged.emit("Listening...")
                time.sleep(1)
                # read the audio data from the default microphone
                try:
                    audio_data = r.listen(source, timeout=5)
                except sr.WaitTimeoutError as e:
                    print("Timeout; {0}".format(e))
                self.messageChanged.emit("Time over")
                time.sleep(1)
                self.messageChanged.emit("Recognizing...")
                time.sleep(1)
                # convert speech to text
                text = r.recognize_google(audio_data, language="en-UK", show_all=True)
                try:
                    self.messageChanged.emit("You said:")
                    self.itemsChanged.emit([])
                    list1 = []
                    if len(text) > 0:
                        print(len(text["alternative"]))
                        for x in range(0, len(text["alternative"])):
                            list1.append(text["alternative"][x]["transcript"])
                        self.itemsChanged.emit(list1)
                    else:
                        time.sleep(1)
                        self.messageChanged.emit("\tNothing")

                except LookupError:
                    time.sleep(1)
                    self.messageChanged.emit("Could not Understand Value")

                except sr.UnknownValueError:
                    time.sleep(1)
                    self.messageChanged.emit(
                        "Voice Recognition could not understand audio"
                    )

                except sr.RequestError as e:
                    time.sleep(1)
                    self.messageChanged.emit(
                        "Voice Recognition could not request results ; {0}".format(e)
                    )
        else:
            time.sleep(1)
            self.messageChanged.emit(
                "NO Internet Connection. \nCheck your Internet Connection"
            )
        time.sleep(1)
        self.messageChanged.emit("Completed")


class Function(QtWidgets.QMainWindow, Ui_Function):
    def __init__(self, parent=None):
        super(Function, self).__init__(parent)
        self.setupUi(self)

        self.worker = SpeechWorker()
        self.worker.messageChanged.connect(self.updateMessage)
        self.worker.itemsChanged.connect(self.updateItems)
     
        self.pushButton.clicked.connect(self.worker.speech)
        self.pushButton_2.clicked.connect(lambda: give_rec(self.lineEdit.text()))
        

    @QtCore.pyqtSlot(str)
    def updateMessage(self, message):
        self.label_5.setText(message)
        self.label_5.adjustSize()

    @QtCore.pyqtSlot(list)
    def updateItems(self, items):
        self.comboBox.clear()
        self.comboBox.addItems(items)
    

    


if __name__ == "__main__":
    import sys
    
    app = QtWidgets.QApplication(sys.argv)
    w = Function()
    w.show()
    sys.exit(app.exec_())

importing Jupyter notebook from Untitled12.ipynb
4144                      Murder in the Smithsonian
376                             An Accidental Woman
460                             Forever Amber Brown
2189                          Nine Princes in Amber
5302    You Can't Eat Your Chicken Pox, Amber Brown
2188                             The Guns of Avalon
6278                                  Hurricane Bay
2152                             The diMedici Bride
3948                    Amber Brown Is Not A Crayon
4029                                Call It Destiny
Name: Original_Title, dtype: object
1017                                                Toxin
9536                             101 Wacky Computer Jokes
1546                                    The Farmer's Wife
7339                                      The Summer Tree
4000                                                  NaN
7046                      Demons Don't Dream (Xanth, #16)
4099                In Rajasthan (Lonely Planet

SystemExit: 0

/home/hina/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
%tb

SystemExit: 0

In [5]:

def give_rec(title,books,sig):
#sig[0]
    indices = pd.Series(books.index, index = books['Original_Title']).drop_duplicates()
    list(enumerate(sig[indices['Two for Texas']]))
    sorted(list(enumerate(sig[indices['Two for Texas']])), key=lambda x: x[1], reverse=True)
    idx=indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = (sig_scores[1:11])
    book_indices = [i[0] for i in sig_scores]
    print(books['Original_Title'].iloc[book_indices])
books,sig=give_rec1()
give_rec("Angel of Hope",books,sig)

4144                      Murder in the Smithsonian
376                             An Accidental Woman
460                             Forever Amber Brown
2189                          Nine Princes in Amber
5302    You Can't Eat Your Chicken Pox, Amber Brown
2188                             The Guns of Avalon
6278                                  Hurricane Bay
2152                             The diMedici Bride
3948                    Amber Brown Is Not A Crayon
4029                                Call It Destiny
Name: Original_Title, dtype: object


In [6]:
give_rec("Kim",books,sig)

1017                                                Toxin
9536                             101 Wacky Computer Jokes
1546                                    The Farmer's Wife
7339                                      The Summer Tree
4000                                                  NaN
7046                      Demons Don't Dream (Xanth, #16)
4099                In Rajasthan (Lonely Planet Journeys)
2332    What Every American Should Know About the Rest...
7017    The New York Diaries: Too-True Tales of Urban ...
4488                                  Midnight's Children
Name: Original_Title, dtype: object


In [ ]:
def give_rec1():
    books = pd.read_csv("book_data.csv",encoding='ISO-8859-1')

    books = books.drop_duplicates()

    from sklearn.feature_extraction.text import TfidfVectorizer

    tfv=TfidfVectorizer(min_df=3, max_features = None,
                    strip_accents = 'unicode', analyzer = 'word', token_pattern=r'\w{1,}',
                    ngram_range = (1,3),
                    stop_words = 'english')

    books['Description']=books['Description'].fillna('')

    tfv_matrix = tfv.fit_transform(books['Description'])
    tfv_matrix
    tfv_matrix.shape
    from sklearn.metrics.pairwise import sigmoid_kernel

    sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
    return (books,sig)